A notebook to explain the function get_data_merged_dynamic_flatten_full(self, target_format="cls", centered_reduced=False)

In [1]:
from transplant.tools.learningset import Learningset
learningset = Learningset()


X_train, X_test, y_train, y_test, X_col = learningset.get_data_merged_dynamic_flatten_full("cls", False)

C:\Users\Delanoue\Documents\GitHubRepo\batch_5_transplant\transplant\tools\learningset.py:133: FutureWarning: 'id_patient' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  grouped_time = X_dyn.groupby(['id_patient'])['time']
C:\Users\Delanoue\Documents\GitHubRepo\batch_5_transplant\transplant\tools\learningset.py:170: FutureWarning: 'id_patient' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  train_grouped=train_dynamic_0.drop(['time'],axis=1).groupby(['id_patient'], as_index=False) #On enregistre les groupes
C:\Users\Delanoue\Documents\GitHubRepo\batch_5_transplant\transplant\tools\learningset.py:171: FutureWarning: 'id_patient' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  test_grouped=test_dynamic_0.drop(['time'],axis=1).groupby(['id_patient'], as_index=Fals

In [2]:
X_col.shape

(182,)

Il faut chopper date début, fin , longeur 

Il faudrait rajouter : 

    -str deviation 
    -min 
    -max
    -30 dernières minutes 
    -etc...
    
    Mais garder en tête que que ça augmente considérablement la taille de nos features 

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import pandas as pd
import numpy as np

from transplant.tools.dataset import Dataset
dataset=Dataset()

Ici on choppe le début , la fin et la longeure de l'opération 

In [5]:
from transplant.tools.learningset import Learningset
learningset = Learningset()

train_static_0, test_static_0 = learningset.get_static_filled()

train_dynamic_0, test_dynamic_0 = dataset.get_dynamic()

In [6]:
train_dynamic_0 = train_dynamic_0.fillna(0)
test_dynamic_0 = test_dynamic_0.fillna(0)

In [56]:
def add_start_end_length_op_to_static(X_stat, X_dyn):
            grouped_time = X_dyn.groupby(['id_patient'])['time']

            time_start_df = grouped_time.first().to_frame()
            time_start_df.columns = ['start_operation']
            time_start_df['id_patient'] = time_start_df.index

            X_return = pd.merge(X_stat, time_start_df, on='id_patient')

            time_ends_df = grouped_time.last().to_frame()
            time_ends_df.columns = ['ends_operation']
            time_ends_df['id_patient'] = time_ends_df.index

            X_return = pd.merge(X_return, time_ends_df, on='id_patient')

            X_return['length_op'] = (X_return['ends_operation'] - X_return['start_operation']).apply(lambda x: x.seconds//60)
            
            X_return['start_operation_year']=X_return['start_operation'].apply(lambda x: x.year)
            X_return['start_operation_month']=X_return['start_operation'].apply(lambda x: x.month)
            X_return['start_operation_day']=X_return['start_operation'].apply(lambda x: x.dayofyear)
            
            
            X_return['ends_operation_year']=X_return['ends_operation'].apply(lambda x: x.year)
            X_return['ends_operation_month']=X_return['ends_operation'].apply(lambda x: x.month)
            X_return['ends_operation_day']=X_return['ends_operation'].apply(lambda x: x.dayofyear)
            
            print(X_return['length_op'].iloc[0])
            print(X_return['ends_operation'].iloc[0])

            return X_return.drop(['length_op','ends_operation','start_operation'], axis=1)


In [57]:
train_static_1 = add_start_end_length_op_to_static(train_static_0, train_dynamic_0)
test_static_1 = add_start_end_length_op_to_static(test_static_0, test_dynamic_0)

473
2013-09-09 07:16:00
1129
2012-06-11 19:53:00


In [58]:
train_static_1.head()

,Age_donor,Aspirations_donor,BMI_donor,Donneur_CPT,Insuffisance_renale,LAS,PAPS,PFO,PF_donor,Poids,...,thoracic_surgery_history,time_on_waiting_liste,transplanted_twice_during_study_period,target,start_operation_year,start_operation_month,start_operation_day,ends_operation_year,ends_operation_month,ends_operation_day
0,41,3.0,25.689562,5550.0,0.0,39.3,68.884058,0.0,457.0,43.0,...,0.0,76.0,0.0,1,2013,9,251,2013,9,252
1,43,2.0,19.531250,4750.0,0.0,31.7,68.884058,0.0,497.0,51.0,...,0.0,47.0,0.0,1,2013,3,64,2013,3,64
2,51,3.0,28.685145,5250.0,0.0,51.9,55.000000,0.0,240.0,55.0,...,0.0,25.0,0.0,0,2013,7,187,2013,7,187
3,34,3.0,31.960227,7000.0,1.0,36.3,38.000000,0.0,152.0,65.0,...,1.0,63.0,0.0,0,2015,11,310,2015,11,311
4,63,0.0,22.892820,4450.0,0.0,37.4,40.000000,0.0,390.0,70.0,...,1.0,17.0,0.0,0,2016,4,93,2016,4,93


In [ ]:
def merge_dyn_sta(X_train_static, X_train_dynamic, X_test_static, X_test_dynamic):
    return pd.merge(X_train_static, X_train_dynamic, on='id_patient'), pd.merge(X_test_static, X_test_dynamic, on='id_patient')


In [ ]:
liste_func=[np.mean,np.std,np.amax,np.amin] #np.median doesn't work ...
liste_func_name=['mean','std','max','min']



In [ ]:
#train_dynamic_0.drop(['time'],axis=1).groupby(['id_patient'], as_index=False).median()

In [ ]:
#train_dynamic_0.drop(['time'],axis=1).groupby(['id_patient'], as_index=False).apply(np.amin)

Avoir le mean

In [ ]:
train_glob_0, test_glob_0 = train_static_1 , test_static_1

for i in range(len(liste_func)) :
    print('On en est à '+liste_func_name[i])
    func=liste_func[i]
    
    train_grouped=train_dynamic_0.drop(['time'],axis=1).groupby(['id_patient'], as_index=False) #On enregistre les groupes
    test_grouped=test_dynamic_0.drop(['time'],axis=1).groupby(['id_patient'], as_index=False)
    
    train_id=train_grouped['id_patient'].apply(np.mean)  #On conserve le id du patient
    test_id=test_grouped['id_patient'].apply(np.mean)
    
   
    train_dynamic_flat = train_grouped.apply(func)   #On applique la fonction
    test_dynamic_flat = test_grouped.apply(func)

    train_dynamic_flat.rename(columns=lambda x: x+'_'+liste_func_name[i] if x!='id_patient' else x, inplace=True)
    test_dynamic_flat.rename(columns=lambda x: x+'_'+liste_func_name[i]  if x!='id_patient' else x, inplace=True)
    
    train_dynamic_flat['id_patient']=train_id
    test_dynamic_flat['id_patient']=test_id
    print(train_glob_0.shape)
    train_glob_0, test_glob_0 = merge_dyn_sta(train_glob_0, train_dynamic_flat, test_glob_0, test_dynamic_flat)
    


In [ ]:
train_glob_0.columns

In [ ]:
target_format = "cls"
centered_reduced = False

In [ ]:


def center_reduce_data(W_train, W_test):
    mean_train = W_train.mean()
    std_train = W_test.std()

    return (W_train-mean_train)/std_train, (W_test-mean_train)/std_train

dic_to_One_Hot = {0: [1, 0], 1: [0, 1]}

y_train_cls = np.array(train_glob_0['target'])
y_train_hot = np.array(list(train_glob_0['target'].map(dic_to_One_Hot)))

y_test_cls = np.array(test_glob_0['target'])
y_test_hot = np.array(list(test_glob_0['target'].map(dic_to_One_Hot)))

if centered_reduced:
    X_train, X_test = center_reduce_data(train_glob_0.drop(['target'], axis=1), test_glob_0.drop(['target'], axis=1))

    X_train = np.array(X_train)
    X_test = np.array(X_test)

else:
    X_train = np.array(train_glob_0.drop(['target'], axis=1))
    X_test = np.array(test_glob_0.drop(['target'], axis=1))

        # Return
if target_format == "cls":
    return X_train, X_test, y_train_cls, y_test_cls, train_glob_0.drop(['target'], axis=1).columns

if target_format == "One_Hot":
    return X_train, X_test, y_train_hot, y_test_hot, train_glob_0.drop(['target'], axis=1).columns
